In [ ]:
import pandas as pd
from datetime import timedelta
from google.colab import drive
!pip install xlsxwriter

drive.mount("/content/drive", force_remount=True)

fileName = "Denumirea setului de date"
fileFolderPath = "Calea folder-ului in care se regaseste setul de date"
excel_path = fileFolderPath + fileName + ".xlsx"
xls = pd.ExcelFile(excel_path)

df_setari_generale = pd.read_excel(xls, sheet_name="SetariGenerale")
df_depozite = pd.read_excel(xls, sheet_name="Depozite")
df_clinici = pd.read_excel(xls, sheet_name="Clinici")
df_strategii = pd.read_excel(xls, sheet_name="Strategii")
df_strategii_medicamente = pd.read_excel(xls, sheet_name="StrategiiMedicamente")
df_vizite = pd.read_excel(xls, sheet_name="Vizite")
df_medicamente = pd.read_excel(xls, sheet_name="Medicamente")
df_dispensari = pd.read_excel(xls, sheet_name="Dispensari")
df_pacienti = pd.read_excel(xls, sheet_name="Pacienti")
df_pacienti_vizite = pd.read_excel(xls, sheet_name="PacientiVizite")
df_loturi = pd.read_excel(xls, sheet_name="Loturi")
df_inventar_clinici = pd.read_excel(xls, sheet_name="InventarClinici")
df_inventar_depozite = pd.read_excel(xls, sheet_name="InventarDepozite")

azi = pd.to_datetime(df_setari_generale.loc[0, "DataCurenta"], format="%d/%m/%Y")

Mounted at /content/drive


In [ ]:
def identifica_pacienti_eligibili(df_clinici, df_pacienti):
    clinici_valide = df_clinici[
        (df_clinici["ClinicaActiva"] == 1) & (df_clinici["AprovizionareActiva"] == 1)
    ]["NumeClinica"]
    pacienti_valizi = df_pacienti[
        (df_pacienti["NumeClinica"].isin(clinici_valide)) &
        (df_pacienti["UltimaVizitaInregistrata"] != "Screening") &
        (df_pacienti["UltimaVizitaInregistrata"] != "Finalizare")
    ]
    return pacienti_valizi

def calculeaza_strategie_data_fereastra_cutoff(pacient, folosesteStrategieZileFereastraMinima):
  strategie_clinica = df_clinici[df_clinici['NumeClinica'] == pacient["NumeClinica"]]["NumeStrategie"].values[0]
  if folosesteStrategieZileFereastraMinima == True:
    strategie_zile_fereastra_cutoff = int(df_strategii[df_strategii["NumeStrategie"] == strategie_clinica]["ZileFereastraMinima"].values[0])
    strategie_data_fereastra_cutoff = pd.to_datetime(azi + timedelta(days=int(strategie_zile_fereastra_cutoff)))
  else:
    strategie_zile_fereastra_cutoff = int(df_strategii[df_strategii["NumeStrategie"] == strategie_clinica]["ZileFereastraMaxima"].values[0])
    strategie_data_fereastra_cutoff = pd.to_datetime(azi + timedelta(days=int(strategie_zile_fereastra_cutoff)))
  return strategie_data_fereastra_cutoff

def proiecteaza_vizite_pacienti(pacienti, vizite, vizitePacienti, folosesteStrategieZileFereastraMinima):
    proiectii = []

    for _, pacient in pacienti.iterrows():
        ultima_vizita = pacient["UltimaVizitaInregistrata"]
        index_ultima_vizita = vizite[ultima_vizita]["IndexVizita"]
        data_ultima = pd.to_datetime(pacient["DataUltimaVizitaInregistrata"], format="%d/%m/%Y")
        strategie_data_fereastra_cutoff = calculeaza_strategie_data_fereastra_cutoff(pacient, folosesteStrategieZileFereastraMinima)
        vizite_viitoare = {
            key: val for key, val in vizite.items()
            if val["IndexVizita"] > index_ultima_vizita
        }

        for numeVizita, vizita in vizite_viitoare.items():
            if pd.notnull(vizita["ZilePrevazute"]):
                vizitaDeBaza = vizita["VizitaDeBaza"]
                index_vizita_baza = vizite[vizitaDeBaza]["IndexVizita"]

                if index_vizita_baza <= index_ultima_vizita:
                    df_filtru = vizitePacienti[
                        (vizitePacienti["NumarPacient"] == pacient["NumarPacient"]) &
                        (vizitePacienti["NumeVizita"] == vizitaDeBaza)
                    ]

                    if not df_filtru.empty:
                        data_vizita_baza = pd.to_datetime(df_filtru.iloc[0]["DataVizita"], format="%d/%m/%Y")
                        data_fereastra_minima = data_vizita_baza + timedelta(days=int(vizita["ZileFereastraMinima"]))
                        data_fereastra_maxima = data_vizita_baza + timedelta(days=int(vizita["ZileFereastraMaxima"]))

                        if data_fereastra_minima <= strategie_data_fereastra_cutoff and data_fereastra_maxima >= azi:
                            proiectii.append({
                                "NumarPacient": pacient["NumarPacient"],
                                "NumeClinica": pacient["NumeClinica"],
                                "Tratament": pacient["Tratament"],
                                "NumeVizita": numeVizita,
                                "DataEstimataVizita": data_vizita_baza + timedelta(days=int(vizita["ZilePrevazute"])),
                                "DataFereastraMinima": data_fereastra_minima,
                                "DataFereastraMaxima": data_fereastra_maxima
                            })
                else:
                    data_vizita_baza_viitor = next(
                        (p["DataEstimataVizita"] for p in proiectii
                         if p["NumarPacient"] == pacient["NumarPacient"] and p["NumeVizita"] == vizitaDeBaza),
                        None
                    )

                    if data_vizita_baza_viitor is not None:
                        data_fereastra_minima = data_vizita_baza_viitor + timedelta(days=int(vizita["ZileFereastraMinima"]))
                        data_fereastra_maxima = data_vizita_baza_viitor + timedelta(days=int(vizita["ZileFereastraMaxima"]))
                        if data_fereastra_minima <= strategie_data_fereastra_cutoff and data_fereastra_maxima >= azi:
                            proiectii.append({
                                "NumarPacient": pacient["NumarPacient"],
                                "NumeClinica": pacient["NumeClinica"],
                                "Tratament": pacient["Tratament"],
                                "NumeVizita": numeVizita,
                                "DataEstimataVizita": data_vizita_baza_viitor + timedelta(days=int(vizita["ZilePrevazute"])),
                                "DataFereastraMinima": data_fereastra_minima,
                                "DataFereastraMaxima": data_fereastra_maxima
                            })

    return pd.DataFrame(proiectii)

def inventar_eligibil_clinici(df_inventar_clinici, df_loturi, df_medicamente, azi):
    inventar = df_inventar_clinici.merge(
        df_loturi[["NumeLot","DataExpirare"]], on="NumeLot", how="left"
    ).merge(
        df_medicamente[["Medicament","DNC"]], on="Medicament", how="left"
    )

    inventar["DataExpirare"] = pd.to_datetime(inventar["DataExpirare"], format="%d/%m/%Y")
    inventar["DNC"] = pd.to_numeric(inventar["DNC"], errors="coerce").fillna(0).astype(int)
    inventar["PragDNC"] = (pd.to_datetime(azi) + pd.to_timedelta(inventar["DNC"], unit="D"))
    inventar_eligibil = inventar[inventar["DataExpirare"] >= inventar["PragDNC"]].copy()
    inventar_eligibil.drop(columns=["PragDNC"], inplace=True)
    return inventar_eligibil

def adauga_necesar_medicamente(df_proiectii, df_dispensari, df_medicamente):
    proiectii_extinse = df_proiectii.merge(
        df_dispensari,
        how="left",
        left_on=["Tratament", "NumeVizita"],
        right_on=["Tratament", "NumeVizita"]
    )

    proiectii_extinse = proiectii_extinse.merge(
        df_medicamente[["Medicament", "DND"]],
        on="Medicament",
        how="left"
    )

    proiectii_extinse["DataMinExpirare"] = proiectii_extinse["DataFereastraMaxima"] + pd.to_timedelta(proiectii_extinse["DND"], unit="D")

    return proiectii_extinse[[
        "NumarPacient",
        "NumeClinica",
        "Tratament",
        "NumeVizita",
        "Medicament",
        "CantitateMedicament",
        "DataEstimataVizita",
        "DataFereastraMinima",
        "DataFereastraMaxima",
        "DataMinExpirare"
    ]]

def genereaza_buffer_din_strategii(df_clinici, df_strategii_medicamente, df_medicamente, azi):
    clinici_active = df_clinici[
        (df_clinici["ClinicaActiva"] == 1) &
        (df_clinici["AprovizionareActiva"] == 1)
    ][["NumeClinica", "NumeStrategie"]]

    bufferuri = clinici_active.merge(
        df_strategii_medicamente,
        on="NumeStrategie",
        how="left"
    )

    bufferuri = bufferuri.rename(columns={"ValoareBuffer": "CantitateMedicament"})
    bufferuri = bufferuri.merge(df_medicamente[["Medicament", "DNC"]], on="Medicament", how="left")
    bufferuri["DataMinExpirare"] = azi + pd.to_timedelta(bufferuri["DNC"], unit="D")
    bufferuri["Tratament"] = "N/A"
    bufferuri["NumeVizita"] = "BUFFER"
    bufferuri["DataEstimataVizita"] = azi
    bufferuri["DataFereastraMinima"] = azi
    bufferuri["DataFereastraMaxima"] = azi
    bufferuri = bufferuri[[
        "NumeClinica", "Tratament", "NumeVizita", "Medicament",
        "CantitateMedicament", "DataEstimataVizita",
        "DataFereastraMinima", "DataFereastraMaxima", "DataMinExpirare"
    ]]

    return bufferuri

def calculeaza_SAI_secvential(df_necesar, df_inventar_clinici, df_loturi):
    loturi_exp = dict(zip(df_loturi["NumeLot"], pd.to_datetime(df_loturi["DataExpirare"], format="%d/%m/%Y")))
    df_inventar = df_inventar_clinici.copy()
    df_inventar["DataExpirare"] = df_inventar["NumeLot"].map(loturi_exp)

    df_inventar = df_inventar.sort_values(by=["NumeClinica", "Medicament", "DataExpirare"])

    df_necesar = df_necesar.sort_values(by=["NumeClinica", "Medicament", "DataMinExpirare"]).copy()

    rezultate = []

    for _, row in df_necesar.iterrows():
        clinica = row["NumeClinica"]
        medicament = row["Medicament"]
        necesar = row["CantitateMedicament"]
        min_exp = row["DataMinExpirare"]

        loturi_eligibile = df_inventar[
            (df_inventar["NumeClinica"] == clinica) &
            (df_inventar["Medicament"] == medicament) &
            (df_inventar["DataExpirare"] >= min_exp) &
            (df_inventar["CantitateMedicament"] > 0)
        ]

        acoperit = 0
        folosite = []

        for i, lot in loturi_eligibile.iterrows():
            disponibil = lot["CantitateMedicament"]
            de_folosit = min(necesar - acoperit, disponibil)

            if de_folosit > 0:
                df_inventar.at[i, "CantitateMedicament"] -= de_folosit
                acoperit += de_folosit
                folosite.append((lot["NumeLot"], de_folosit))

            if acoperit >= necesar:
                break

        rezultate.append({
            "NumeClinica": clinica,
            "Medicament": medicament,
            "CantitateNecesara": necesar,
            "CantitateAcoperita": acoperit,
            "CantitateNeacoperita": max(0, necesar - acoperit),
            "DataMinExpirare": min_exp,
            "FullyCovered": acoperit >= necesar,
            "LoturiFolosite": folosite
        })

    return pd.DataFrame(rezultate)

def genereaza_shipments_din_depozit(
    df_comenzi, df_inventar_depozite, df_loturi, df_depozite
):
    loturi_exp = dict(zip(df_loturi["NumeLot"], pd.to_datetime(df_loturi["DataExpirare"], format="%d/%m/%Y")))
    df_inventar = df_inventar_depozite.copy()
    df_inventar["DataExpirare"] = df_inventar["NumeLot"].map(loturi_exp)

    df_inventar = df_inventar.sort_values(by=["NumeDepozit", "Medicament", "DataExpirare"])

    shipments = []

    for _, comanda in df_comenzi.iterrows():
        clinica = comanda["NumeClinica"]
        medicament = comanda["Medicament"]
        necesar = comanda["CantitateDeTrimis"]
        min_exp = comanda["DataMinExpirare"]

        depozite_eligibile = df_depozite[
            df_depozite["CliniciAprovizionate"].str.contains(clinica)
        ]["NumeDepozit"].tolist()

        acoperit = 0

        for depozit in depozite_eligibile:
            loturi_disponibile = df_inventar[
                (df_inventar["NumeDepozit"] == depozit) &
                (df_inventar["Medicament"] == medicament) &
                (df_inventar["DataExpirare"] >= min_exp) &
                (df_inventar["CantitateMedicament"] > 0)
            ]

            for i, lot in loturi_disponibile.iterrows():
                disponibil = lot["CantitateMedicament"]
                de_livrat = min(necesar - acoperit, disponibil)

                if de_livrat > 0:
                    df_inventar.at[i, "CantitateMedicament"] -= de_livrat
                    acoperit += de_livrat

                    shipments.append({
                        "NumeClinica": clinica,
                        "Medicament": medicament,
                        "CantitateLivrata": de_livrat,
                        "NumeLot": lot["NumeLot"],
                        "DataExpirare": lot["DataExpirare"],
                        "NumeDepozit": depozit
                    })

                if acoperit >= necesar:
                    break
            if acoperit >= necesar:
                break

    return pd.DataFrame(shipments)

def actualizeaza_inventar_depozite(df_inventar_depozite, df_shipments):
    df_inventar = df_inventar_depozite.copy()

    for _, row in df_shipments.iterrows():
        mask = (
            (df_inventar["NumeDepozit"] == row["NumeDepozit"]) &
            (df_inventar["NumeLot"] == row["NumeLot"]) &
            (df_inventar["Medicament"] == row["Medicament"])
        )
        if mask.any():
            idx = df_inventar[mask].index[0]
            df_inventar.at[idx, "CantitateMedicament"] = max(
                0,
                df_inventar.at[idx, "CantitateMedicament"] - row["CantitateLivrata"]
            )
    return df_inventar

In [ ]:
pacienti_eligibili = identifica_pacienti_eligibili(df_clinici, df_pacienti)

df_vizite["IndexVizita"] = pd.to_numeric(df_vizite["IndexVizita"], errors="coerce")
df_vizite = df_vizite.sort_values(by="IndexVizita")
df_vizite_proiectate_min = proiecteaza_vizite_pacienti(pacienti_eligibili, df_vizite.set_index(df_vizite.columns[0]).to_dict(orient="index"), df_pacienti_vizite, True)

df_necesar_pacienti = adauga_necesar_medicamente(df_vizite_proiectate_min, df_dispensari, df_medicamente)

df_buffer_min = genereaza_buffer_din_strategii(
    df_clinici,
    df_strategii_medicamente,
    df_medicamente,
    azi
)

df_total_necesar = pd.concat([df_necesar_pacienti, df_buffer_min], ignore_index=True)

df_inventar_eligibil_clinici = inventar_eligibil_clinici(
    df_inventar_clinici,
    df_loturi,
    df_medicamente,
    azi
)

df_total_necesar_standard = pd.concat([df_necesar_pacienti, df_buffer_min], ignore_index=True)

df_necesar_agregat = df_total_necesar_standard.groupby(
    ["NumeClinica", "Medicament", "DataMinExpirare"], as_index=False
)["CantitateMedicament"].sum()

df_rezultat_sai = calculeaza_SAI_secvential(df_necesar_agregat, df_inventar_eligibil_clinici, df_loturi)

clinici_cu_resupply_standard = df_rezultat_sai[df_rezultat_sai["FullyCovered"] == False]["NumeClinica"].unique()

if len(clinici_cu_resupply_standard) == 0:
  print("✅Toate clinicile sunt complet acoperite în fereastra minimă deci NU este necesara extinderea calculului in fereasta maximă✅")
  with pd.ExcelWriter(fileFolderPath + "RezultateAlgoritm_" + fileName + ".xlsx", engine="xlsxwriter") as writer:
    df_rezultat_sai.to_excel(writer, sheet_name="Fereastra_Minima", index=False)
  print("✅Rezultatele au fost incarcate.✅")
else:
  print("❌Exista cel putin o clinica care nu este complet acoperita în fereastra minimă deci este necesara extinderea calculului in fereasta maximă❌")
  df_comenzi_resupply_final = pd.DataFrame(columns=["NumeClinica", "Medicament", "CantitateDeTrimis", "DataMinExpirare"])
  pacienti_pentru_predictiv = pacienti_eligibili[pacienti_eligibili["NumeClinica"].isin(clinici_cu_resupply_standard)].copy()
  df_vizite_proiectate_max = proiecteaza_vizite_pacienti(
      pacienti_pentru_predictiv,
      df_vizite.set_index(df_vizite.columns[0]).to_dict(orient="index"),
      df_pacienti_vizite,
      folosesteStrategieZileFereastraMinima=False
  )

  df_necesar_pacienti_predictiv = adauga_necesar_medicamente(
      df_vizite_proiectate_max,
      df_dispensari,
      df_medicamente
  )

  df_buffer_max = df_buffer_min[df_buffer_min["NumeClinica"].isin(clinici_cu_resupply_standard)].copy()

  df_total_necesar_final = pd.concat(
      [df_necesar_pacienti_predictiv, df_buffer_max],
      ignore_index=True
  )

  df_total_necesar_final = df_total_necesar_final.groupby(
      ["NumeClinica", "Medicament", "DataMinExpirare"], as_index=False
  )["CantitateMedicament"].sum()

  df_rezultat_final = calculeaza_SAI_secvential(
      df_total_necesar_final,
      df_inventar_eligibil_clinici,
      df_loturi
  )

  df_comenzi_resupply_final = df_rezultat_final[df_rezultat_final["CantitateNeacoperita"] > 0].copy()
  df_comenzi_resupply_final = df_comenzi_resupply_final.rename(columns={
      "CantitateNeacoperita": "CantitateDeTrimis"
  })[[
      "NumeClinica", "Medicament", "CantitateDeTrimis", "DataMinExpirare"
  ]]
  df_shipments_generate = genereaza_shipments_din_depozit(
      df_comenzi_resupply_final,
      df_inventar_depozite,
      df_loturi,
      df_depozite
  )

  df_inventar_depozite_post = actualizeaza_inventar_depozite(df_inventar_depozite, df_shipments_generate)

  with pd.ExcelWriter(fileFolderPath + fileName + "_Rezultate_Algoritm.xlsx", engine="xlsxwriter") as writer:
    df_rezultat_sai.to_excel(writer, sheet_name="Fereastra_Minima", index=False)
    df_rezultat_final.to_excel(writer, sheet_name="Fereastra_Maxima", index=False)
    df_comenzi_resupply_final.to_excel(writer, sheet_name="Comenzi_Reaprovizionare", index=False)
    df_shipments_generate.to_excel(writer, sheet_name="Comenzi_Generate", index=False)
    df_inventar_depozite_post.to_excel(writer, sheet_name="Inventar_Depozite", index=False)

  print("✅Reaprovizionarea s-a terminat. Rezultatele au fost incarcate.✅")

✅Toate clinicile sunt complet acoperite în fereastra minimă deci NU este necesara extinderea calculului in fereasta maximă✅
✅Rezultatele au fost incarcate.✅
